Testnet here: https://testnet.binancefuture.com/en/futures/BTCUSDT

In [2]:
api_key = "6ce63f3406fd8ebbff01054a66c25fe3c851c45932088c8ca3131a7005188462"
secret_key = "aa3ea32929252467fa5ffeac5818c95beabfb5dba691ef445e7eaa31ea0d15f6"

In [3]:
from lib.TechnicalIndicators import *
from binance.client import Client
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [1]:
class FuturesTrader():
    def __init__(self, symbol="btcusd", testnet = True):
        ####API CONNECTIONS ####
        self.stream = None
        self.client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = testnet)
        #######################
        self.data = None
        self.symbol = symbol
        self.asset = self.get_asset(self.symbol) #get asset like "USDT"
        #### "BTCUSD" is not valid with official api methods, need to use "BTCUSDT" ####
        symbol = self.symbol + "t" if self.symbol.endswith('usd') else self.symbol
        self.symbol_upper = symbol.upper()
        self.strategies = [] #this stores the strategies used
        self.open_orders = []
        self.leverage = 1 #stores current leverage
        self.current_pos = 0 #stores current position
        self.ind_sum = 0 #stores indicators sum
        self.initial_balance = self.get_current_balance() #sotres the initial balance of the session
        self.available_balance = self.initial_balance #stores available balance
        self.last_close_price = 0 #stores last close price
    
    def get_asset(self, symbol):
        if symbol.endswith('busd'): return "BUSD"
        if symbol.endswith('usd'): return "USDT"
        if symbol.endswith('eth'): return "ETH"
        if symbol.endswith('bnb'): return "BNB"
        if symbol.endswith('btc'): return "BTC"
        
    def message_handler(self, msg):
        if 'result' in msg.keys(): #skip first message
            return
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        col_num = self.data.shape[1]
        self.data.loc[start_time] = [first, high, low, close, volume, complete] + [False]*(col_num-6)
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.last_close_price = close
            #print("candle completed", end="")
            self.run_strategy()
        
    def start_streaming(self, interval="1m"):
        self.stream = CMFuturesWebsocketClient()
        self.stream.start()
        self.stream.kline(
            symbol=self.symbol.lower() +"_perp",
            id=2,
            interval=interval,
            callback=self.message_handler,
        )
        
    def stop_streaming(self):
        self.stream.stop()
        
    def get_most_recent_data(self, num_candles=100, interval = "1m"):
        #### Get start time for candles ####
        now = datetime.utcnow()
        past = str(now - self.available_intervals(num_candles)[interval])
        #### Request candles and prepare the df ####
        bars = self.client.futures_historical_klines(symbol = self.symbol_upper.lower(), 
                                        interval = interval, 
                                        start_str =past,
                                        end_str = None)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Close Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]    
        self.data = df
    
    def start_trading(self, num_candles = 100, interval = "1m"):
        if interval in self.available_intervals(num_candles).keys():
            self.get_most_recent_data(num_candles = num_candles, interval=interval)
            self.prepare_strategies()
            self.start_streaming(interval)
        else:
            print("That interval is not available")
            
    def stop_trading(self):
        self.stop_streaming()
        self.go_neutral()
        #print ending metrics here!!
        
    def available_intervals(self, candles_required):
        '''
        Helper function for "get_most_recent_data" method.
        
        '''
        return {
            "1m"  : timedelta(minutes=candles_required),
            "3m"  : timedelta(minutes=candles_required*3),
            "5m"  : timedelta(minutes=candles_required*5),
            "15m" : timedelta(minutes=candles_required*15),
            "30m" : timedelta(minutes=candles_required*30),
            "1h"  : timedelta(hours=candles_required),
            "2h"  : timedelta(hours=candles_required*2),
            "4h"  : timedelta(hours=candles_required*4),
            "6h"  : timedelta(hours=candles_required*6),
            "8h"  : timedelta(hours=candles_required*8),
            "12h" : timedelta(hours=candles_required*12),
            "1d"  : timedelta(days=candles_required),
            "3d"  : timedelta(days=candles_required*3),
            "1w"  : timedelta(days=candles_required*7),
            "1M"  : timedelta(days=candles_required*28) #this may give less than the desired candles because each month has different amount of days
        }
    def prepare_strategies(self):
        # prepare params
        #SMA
        SMA_S = 2
        SMA_L = 5
        #EWMA
        approx_avg_period_s = 2
        approx_avg_period_l = 5
        #BBS
        dev = 1
        periods = 50
        #prepare strategies
        self.strategies = [
            SMA(
                data = self.data,
                SMA_S = SMA_S,
                SMA_L = SMA_L,
                column = "Close",
                default_strategy = 1
            ),
            EWMA(
                data = self.data,
                approx_avg_period_s = approx_avg_period_s,
                approx_avg_period_l = approx_avg_period_l,
                column = "Close",
                default_strategy = 1
            ),
            BollingerBands(
                data = self.data,
                dev = dev, 
                periods = periods,
                column = "Close",
                default_strategy = 1
            )
        ]
        for strategy in self.strategies:
            strategy.calculate() #add columns to data 
        #self.data.dropna(inplace = True) #dropna after calculating the strategy
    
    def leverage_strategy(self, ind_sum, ind_count, min_lev = 1, max_lev = 10):
        if max_lev < min_lev:
            print("max leverage is less than min leverage")
            return -1
        #y2 = y1 + m(x2-x1) , m =(y2-y1)/(x2-x1)
        #two points of the line: (ind_sum = 1, min_lev) and (ind_sum = ind_count, max_lev)
        ind_sum = abs(ind_sum) #short or long, we want just abs number
        m = (max_lev-min_lev)/(ind_count-1)
        new_leverage = round( min_lev + m*( ind_sum - 1 ))
        self.client.futures_change_leverage(symbol = self.symbol_upper, leverage = new_leverage)
        self.leverage = new_leverage
        return new_leverage
    
    def run_strategy(self):
        self.ind_sum = 0
        for strategy in self.strategies:
            strategy.calculate_for_last_row()
            self.ind_sum += strategy.strategy(-1)
        self.predicted_pos = np.sign(self.ind_sum)
        print(" |pp:"+str(self.predicted_pos)+str("| "), end="")
        if self.predicted_pos == 1 and self.get_position() in [0, -1]:
            self.go_long(prc = True, amount = 95) # go long with full amount
        if self.predicted_pos == -1 and self.get_position() in [0, 1]:
            self.go_short(prc = True, amount = 95) # go short with full amount
        #if self.predicted_pos == 0:
            #self.go_neutral()
            
    def go_long(self, prc = True, amount = None):
        if self.get_position() != 0:
            self.go_neutral() #if some position, go neutral first
        if self.should_end_session(): return
        self.leverage_strategy(ind_sum = self.ind_sum, ind_count = len(self.strategies))
        if prc == True: 
            amount = (self.available_balance * amount/100) * self.leverage
        else:
            amount = amount * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  round(amount/self.last_close_price, 3)
        self.create_order(side = "BUY", quantity = quantity)

    def go_short(self, prc = True, amount = None):
        if self.get_position() != 0:
            self.go_neutral() #if some position, go neutral first
        if self.should_end_session(): return
        self.leverage_strategy(ind_sum = self.ind_sum, ind_count = len(self.strategies))
        if prc == True: 
            amount = (self.available_balance * amount/100) * self.leverage
        else:
            amount = amount * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  round(amount/self.last_close_price, 3)
        self.create_order(side = "SELL", quantity = quantity)
    
    def go_neutral(self):
        if self.get_position() == 1: #if long, sell all
            self.create_order(side = "SELL", close_pos = True)
        elif self.get_position() == -1: #if short, buy all
            self.create_order(side = "BUY", close_pos = True)
    
    def create_order(self, side = "BUY", quantity = 0, close_pos = False):
        if close_pos:
            open_order = self.open_orders[0]
            quantity = open_order["executedQty"]
            order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
                                         type = "MARKET", quantity = quantity)
            order = self.client.futures_get_order(symbol = self.symbol_upper, orderId = order_open["orderId"])
            self.open_orders = []
            self.current_pos = 0
            self.available_balance = self.get_current_balance()
            print(" |CP| ", end="")
            return 0
        order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
                                         type = "MARKET", quantity = quantity)
        order = self.client.futures_get_order(symbol = self.symbol_upper, orderId = order_open["orderId"])
        self.open_orders.append(order)
        self.current_pos = 1 if side == "BUY" else -1
        print(" |OP| ", end="")
        return 0
    
    def get_position(self):
        return self.current_pos
    
    def should_end_session(self):
        if self.available_balance < self.initial_balance * 0.3 and self.get_position() == 0: #no money and no positions
            self.stop_trading()
            return True
    def get_current_balance(self):
        balance = pd.DataFrame(self.client.futures_account_balance())# Asset Balance details
        balance = float(balance[ balance["asset"] == self.asset ].iloc[0]["balance"])
        return balance
            
            

In [4]:
trader = FuturesTrader(symbol="btcusd", testnet = True)

In [5]:
trader.start_trading(interval = "1m", num_candles = 500)

.......... |pp:1| 

Unhandled Error
Traceback (most recent call last):
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 96, in callWithLogger
    return callWithContext({"system": lp}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 80, in callWithContext
    return context.call({ILogContext: newCtx}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 117, in callWithContext
    return self.currentContext().callWithContext(ctx, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 82, in callWithContext
    return func(*args, **kw)
--- <exception caught here> ---
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py", line 683, in _doReadOrWrite
    why = selectable.doRead()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/interne

In [6]:
trader.stop_trading()

In [7]:
trader.data

,Open,High,Low,Close,Volume,Complete,Close|SMA|50,Close|SMA|200,Close|EWMA|2,Close|EWMA|5,Close|BBs|1|50|Lower,Close|BBs|1|50|Upper,Close|BBs|1|50|Distance
Date,,,,,,,,,,,,,
2022-12-05 17:35:00,17071.8,17077.6,17067.6,17077.6,70.334,True,NaN,NaN,17077.6,17077.6,NaN,NaN,NaN
2022-12-05 17:36:00,17078.1,17081.8,17075.9,17079.8,132.173,True,NaN,NaN,17079.066667,17079.433333,NaN,NaN,NaN
2022-12-05 17:37:00,17079.4,17084.1,17077.2,17080.1,91.930,True,NaN,NaN,17079.657143,17079.970968,NaN,NaN,NaN
2022-12-05 17:38:00,17080.1,17081.6,17072.5,17076.7,144.392,True,NaN,NaN,17078.08,17077.35,NaN,NaN,NaN
2022-12-05 17:39:00,17076.4,17083.4,17074.6,17080.5,156.495,True,NaN,NaN,17079.329032,17079.870807,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-05 22:35:00,16948.3,16955.4,16948.3,16955.4,6054.000,True,16954.862,16940.0895,16952.294376,16954.168868,16940.77423,16968.94977,0.538
2022-12-05 22:36:00,16955.4,16955.4,16955.3,16955.3,1528.000,True,16954.62,16939.9255,16953.797188,16955.073774,16940.648557,16968.591443,0.68
2022-12-05 22:37:00,16955.4,16955.4,16955.3,16955.3,711.000,True,16954.25,16939.9455,16954.548594,16955.254755,16940.554622,16967.945378,1.05


In [ ]:
trader.data.to_csv("data/main.csv")

In [7]:
trader.asset

'USDT'

In [6]:
trader.get_position()

1

..candle completed.........candle completed.......candle completed....candle completed.

### cerca de las 12 horas dio este error:
Unhandled Error
Traceback (most recent call last):
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 96, in callWithLogger
    return callWithContext({"system": lp}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 80, in callWithContext
    return context.call({ILogContext: newCtx}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 117, in callWithContext
    return self.currentContext().callWithContext(ctx, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 82, in callWithContext
    return func(*args, **kw)
--- <exception caught here> ---
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py", line 683, in _doReadOrWrite
    why = selectable.doRead()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py", line 248, in doRead
    return self._dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py", line 253, in _dataReceived
    rval = self.protocol.dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py", line 330, in dataReceived
    self._flushReceiveBIO()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py", line 296, in _flushReceiveBIO
    ProtocolWrapper.dataReceived(self, bytes)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/policies.py", line 110, in dataReceived
    self.wrappedProtocol.dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 348, in dataReceived
    self._dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1243, in _dataReceived
    self.consumeData()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1255, in consumeData
    while self.processData() and self.state != WebSocketProtocol.STATE_CLOSED:
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1619, in processData
    fr = self.onFrameEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1747, in onFrameEnd
    self._onMessageEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 384, in _onMessageEnd
    self.onMessageEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 647, in onMessageEnd
    self._onMessage(payload, self.message_is_binary)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 387, in _onMessage
    self.onMessage(payload, isBinary)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/websocket/binance_client_protocol.py", line 30, in onMessage
    self.factory.callback(payload_obj)
  File "/tmp/ipykernel_16254/1118553294.py", line 52, in message_handler
    self.run_strategy()
  File "/tmp/ipykernel_16254/1118553294.py", line 182, in run_strategy
    self.go_short(prc = True, amount = 95) # go short with full amount
  File "/tmp/ipykernel_16254/1118553294.py", line 201, in go_short
    self.go_neutral() #if some position, go neutral first
  File "/tmp/ipykernel_16254/1118553294.py", line 214, in go_neutral
    self.create_order(side = "SELL", close_pos = True)
  File "/tmp/ipykernel_16254/1118553294.py", line 222, in create_order
    order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 5985, in futures_create_order
    return self._request_futures_api('post', 'order', True, data=params)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 339, in _request_futures_api
    return self._request(method, uri, signed, True, **kwargs)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 315, in _request
    return self._handle_response(self.response)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 324, in _handle_response
    raise BinanceAPIException(response, response.status_code, response.text)
binance.exceptions.BinanceAPIException: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.

WARNING:root:WebSocket connection closed: connection was closed uncleanly ("peer dropped the TCP connection without previous WebSocket closing handshake"), code: 1006, clean: False, reason: connection was closed uncleanly ("peer dropped the TCP connection without previous WebSocket closing handshake")
ERROR:root:Lost connection to Server. Reason: [Failure instance: Traceback: <class 'binance.exceptions.BinanceAPIException'>: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py:80:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py:117:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py:82:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py:696:_doReadOrWrite
--- <exception caught here> ---
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py:683:_doReadOrWrite
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py:248:doRead
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py:253:_dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py:330:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py:296:_flushReceiveBIO
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/policies.py:110:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:348:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1243:_dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1255:consumeData
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1619:processData
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1747:onFrameEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:384:_onMessageEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:647:onMessageEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:387:_onMessage
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/websocket/binance_client_protocol.py:30:onMessage
/tmp/ipykernel_16254/1118553294.py:52:message_handler
/tmp/ipykernel_16254/1118553294.py:182:run_strategy
/tmp/ipykernel_16254/1118553294.py:201:go_short
/tmp/ipykernel_16254/1118553294.py:214:go_neutral
/tmp/ipykernel_16254/1118553294.py:222:create_order
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:5985:futures_create_order
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:339:_request_futures_api
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:315:_request
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:324:_handle_response
]. Retrying: 1